In [1]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
import math
import random
import numpy as np
from scipy.optimize import minimize

In [15]:
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
def decompose(a):
    gates = [I, X, Y, Z]
    outlist = []
    for i in range((a.shape)[0]):
        for j in range((a.shape)[0]):
            inlist = [np.trace(a*np.kron(gates[i], gates[j]))/4, np.kron(gates[i], gates[j])]
            outlist.append(inlist)
    return outlist
a = np.asmatrix('1 1 1 1;2 3 4 5; 5 0 4 0; 3 -2 0 9')
b = np.asmatrix(decompose(a))
print(b, b.shape)

[[4.25
  matrix([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])]
 [0.75
  matrix([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0]])]
 [-0.25j
  matrix([[0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
        [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
        [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j]])]
 [-1.75
  matrix([[ 1,  0,  0,  0],
        [ 0, -1,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0, -1]])]
 [2.25
  matrix([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0]])]
 [2.0
  matrix([[0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]])]
 [-1.5j
  matrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
        [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
        [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
        [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]])]
 [0.75
  matrix([[ 0,  0,  1,  0],
        [ 0,  0,  0, -1],
        [ 1,  0,  0,  0],
        [ 0, -1,  0,  0]])]
 [0.75j
  mat

In [6]:
def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    circ.cz(qubits[0], qubits[1])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[1][iz], qubits[iz])

    circ.cz(qubits[1], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[2][iz], qubits[iz])

        ┌───────┐   ┌───────┐   ┌───────┐
q_0: |0>┤ Ry(1) ├─■─┤ Ry(1) ├─■─┤ Ry(1) ├
        ├───────┤ │ ├───────┤ │ ├───────┤
q_1: |0>┤ Ry(1) ├─■─┤ Ry(1) ├─■─┤ Ry(1) ├
        └───────┘   └───────┘   └───────┘


In [9]:
def had_test(gate_type, qubits, ancilla_index, parameters):

    circ.h(ancilla_index)

    apply_fixed_ansatz(qubits, parameters)

    for ie in range (0, len(gate_type[0])):
        if (gate_type[0][ie] == 1):
            circ.cz(ancilla_index, qubits[ie])

    for ie in range (0, len(gate_type[1])):
        if (gate_type[1][ie] == 1):
            circ.cz(ancilla_index, qubits[ie])
    
    circ.h(ancilla_index)
circ = QuantumCircuit(3)
had_test([[0, 0], [0, 1]], [1, 2], 0, [[1, 1], [1, 1], [1, 1]])
print(circ)

          ┌───┐                             ┌───┐
q_0: |0>──┤ H ├───────────────────────────■─┤ H ├
        ┌─┴───┴─┐   ┌───────┐   ┌───────┐ │ └───┘
q_1: |0>┤ Ry(1) ├─■─┤ Ry(1) ├─■─┤ Ry(1) ├─┼──────
        ├───────┤ │ ├───────┤ │ ├───────┤ │      
q_2: |0>┤ Ry(1) ├─■─┤ Ry(1) ├─■─┤ Ry(1) ├─■──────
        └───────┘   └───────┘   └───────┘        


In [13]:
def control_fixed_ansatz(qubits, parameters, ancilla, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, ancilla), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(ancilla, qubits[1], 3)
    circ.cz(qubits[0], 3)
    circ.ccx(ancilla, qubits[1], 3)
    

    for i in range (0, len(qubits)):
        circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, ancilla), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(ancilla, qubits[0], 3)
    circ.cz(qubits[1], 3)
    circ.ccx(ancilla, qubits[0], 3)

    for i in range (0, len(qubits)):
        circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, ancilla), qiskit.circuit.Qubit(reg, qubits[i]))
q_reg = QuantumRegister(4)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1, 2], [[1, 1], [1, 1], [1, 1]], 0, q_reg)
print(circ)

                                                                            »
q3_0: |0>─────────────────■────────────────────■────■────────────────────■──»
         ┌─────────────┐┌─┴─┐┌──────────────┐┌─┴─┐  │                    │  »
q3_1: |0>┤ U3(0.5,0,0) ├┤ X ├┤ U3(-0.5,0,0) ├┤ X ├──┼────────────────────┼──»
         ├─────────────┤└───┘└──────────────┘└───┘┌─┴─┐┌──────────────┐┌─┴─┐»
q3_2: |0>┤ U3(0.5,0,0) ├──────────────────────────┤ X ├┤ U3(-0.5,0,0) ├┤ X ├»
         └─────────────┘                          └───┘└──────────────┘└───┘»
q3_3: |0>───────────────────────────────────────────────────────────────────»
                                                                            »
«                                                                           »
«q3_0: ──■───────■────────────────────────■─────────────────────────■────■──»
«        │       │  ┌─────────────┐     ┌─┴─┐     ┌──────────────┐┌─┴─┐  │  »
«q3_1: ──┼───■───┼──┤ U3(0.5,0,0) ├─────┤ X ├─────┤ U3(-0.5,0,0)

In [14]:
def control_b(ancilla, qubits):

    for ia in qubits:
        circ.ch(ancilla, ia)
circ = QuantumCircuit(3)
control_b(0, [1, 2])
print(circ)

                  
q_0: |0>──■────■──
        ┌─┴─┐  │  
q_1: |0>┤ H ├──┼──
        └───┘┌─┴─┐
q_2: |0>─────┤ H ├
             └───┘


In [ ]:
def special_had_test(gate_type, qubits, ancilla_index, parameters, reg):

    circ.h(ancilla_index)

    control_fixed_ansatz(qubits, parameters, ancilla_index, reg)

    '''for ty in range (0, len(gate_type)):
        if (gate_type[ty] == 1):
            circ.cz(ancilla_index, qubits[ty])'''
    

    control_b(ancilla_index, qubits)
    
    circ.h(ancilla_index)


In [ ]:
def calculate_cost_function(parameters):
    
    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9]]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(5)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('statevector_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1, 2, 3], 0, parameters)

            job = execute(circ, backend)

            result = job.result()
            outputstate = result.get_statevector(circ, decimals=100)
            o = outputstate

            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = float(o[l])**2
                    m_sum+=n

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(5)
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('statevector_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                job = execute(circ, backend)

                result = job.result()
                outputstate = result.get_statevector(circ, decimals=100)
                o = outputstate

                m_sum = 0
                for l in range (0, len(o)):
                    if (l%2 == 1):
                        n = float(o[l])**2
                        m_sum+=n
                mult = mult*(1-(2*m_sum))

            overall_sum_2+=multiply*mult
            
    print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)